In [2]:
#!/usr/bin/env python3
import os
import re
from pathlib import Path

# --------------------------------------------------------------------
# 1. Thay đường dẫn cho phù hợp
# --------------------------------------------------------------------
SMOOTH_ROUTES_DIR   = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-routes"
OUTPUT_INTER_ROUTES = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/new-strategy/routes_inter_all.xml"

# --------------------------------------------------------------------
# 2. Biến toàn cục để track đã seen ID
# --------------------------------------------------------------------
seen_vtype = set()
seen_route = set()

# Pattern để phát hiện vType id="..."
_pattern_vtype = re.compile(r'<\s*vType\b[^>]*\bid\s*=\s*"([^"]+)"')
# Pattern để phát hiện route id="..."
_pattern_route = re.compile(r'<\s*route\b[^>]*\bid\s*=\s*"([^"]+)"')
# Pattern để phát hiện vehicle with fromTaz="X" toTaz="Y"
# Dùng non-greedy để bắt đúng giá trị
_pattern_vehicle = re.compile(
    r'<\s*vehicle\b[^>]*\bfromTaz\s*=\s*"([^"]+)"[^>]*\btoTaz\s*=\s*"([^"]+)"'
)

def merge_inter_routes_linebased():
    # Mở file output
    os.makedirs(os.path.dirname(OUTPUT_INTER_ROUTES), exist_ok=True)
    fout = open(OUTPUT_INTER_ROUTES, "w", encoding="utf-8")

    # 1) Ghi header <routes>
    fout.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    fout.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"\n')
    fout.write('        xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')

    # 2) Duyệt từng file trong SMOOTH_ROUTES_DIR
    for file_path in sorted(Path(SMOOTH_ROUTES_DIR).iterdir(), key=lambda p: p.name):
        name = file_path.name
        # Chỉ lấy file routes_*.xml, không lấy routes_*.alt.xml hoặc routes_*.log
        if not name.startswith("routes_") or not name.endswith(".xml") or name.endswith(".alt.xml"):
            continue

        with open(file_path, "r", encoding="utf-8") as fin:
            for raw_line in fin:
                line = raw_line.strip()
                if not line:
                    continue

                # 2.1. Nếu là <vType ... id="VID" .../>
                if line.startswith("<vType"):
                    m = _pattern_vtype.search(line)
                    if m:
                        vid = m.group(1)
                        if vid not in seen_vtype:
                            seen_vtype.add(vid)
                            fout.write(f"    {line}\n")
                    continue

                # 2.2. Nếu là <route id="RID" .../>
                if line.startswith("<route"):
                    m = _pattern_route.search(line)
                    if m:
                        rid = m.group(1)
                        if rid not in seen_route:
                            seen_route.add(rid)
                            fout.write(f"    {line}\n")
                    continue

                # 2.3. Nếu là <vehicle ... fromTaz="X" ... toTaz="Y" .../>
                if line.startswith("<vehicle"):
                    m = _pattern_vehicle.search(line)
                    if m:
                        from_taz = m.group(1)
                        to_taz   = m.group(2)
                        if from_taz != to_taz:
                            fout.write(f"    {line}\n")
                    continue

                # Mọi dòng khác (routeDistribution, flow, comment, log, alt, ...) bỏ qua

    # 3) Đóng thẻ </routes>
    fout.write("</routes>\n")
    fout.close()
    print(f"Đã gộp xong routes_inter_all.xml tại: {OUTPUT_INTER_ROUTES}")


if __name__ == "__main__":
    merge_inter_routes_linebased()



Đã gộp xong routes_inter_all.xml tại: /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/new-strategy/routes_inter_all.xml


In [ ]:
#!/usr/bin/env python3
import os
import csv
import traci
from sumolib import checkBinary
import xml.etree.ElementTree as ET

# --------------------------------------------------------------------
# 1. Cấu hình các đường dẫn (điều chỉnh cho đúng chỗ của bạn)
# --------------------------------------------------------------------
NET_XML          = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml"
VTYPES_XML       = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/additional-files/vtypes/eidm-1.xml"
ROUTES_INTER_ALL = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/new-strategy/routes_inter_all.xml"
TAZ_XML          = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/taz.xml"
CROSSING_LOG_CSV = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/new-strategy/crossing_log.csv"

os.environ["LD_LIBRARY_PATH"] = os.path.expanduser("~/Libs/libnsl")
os.environ["SUMO_HOME"] = os.path.expanduser("~/Envs/sumo-env/lib/python3.10/site-packages/sumo")

SIM_END          = 24 * 3600   # Giả sử chạy đủ 24h

# --------------------------------------------------------------------
# 2. Hàm đọc TAZ_XML, build edge_to_taz
# --------------------------------------------------------------------
def load_edge_to_taz(taz_file: str) -> dict[str, str]:
    """
    Đọc TAZ_XML, tạo dict { edgeID: tazID } từ <tazSource id="..."/> và <tazSink id="..."/>.
    Giả định <taz id="Z">, mỗi <tazSource id="E"/> hoặc <tazSink id="E"/> chỉ ra
    rằng edge E được xem là 'biên' (entry/exit) của zone Z (và ta gán region = Z).
    """
    edge_to_taz: dict[str, str] = {}
    tree = ET.parse(taz_file)
    root = tree.getroot()
    for taz in root.findall(".//taz"):
        zid = taz.get("id")
        if zid is None:
            continue
        for src in taz.findall("tazSource"):
            eid = src.get("id")
            if eid:
                edge_to_taz[eid] = zid
        for sink in taz.findall("tazSink"):
            eid = sink.get("id")
            if eid:
                edge_to_taz[eid] = zid
    print(f"[INFO] load_edge_to_taz: Đã đọc {len(edge_to_taz)} biên từ TAZ_XML")
    return edge_to_taz

# --------------------------------------------------------------------
# 3. Chạy Inter-region Simulation, ghi crossing_log.csv
# --------------------------------------------------------------------
def run_inter_region():
    edge_to_taz = load_edge_to_taz(TAZ_XML)

    # Mở CSV để ghi header
    with open(CROSSING_LOG_CSV, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            "vehID",      # ID gốc của vehicle
            "departTime", # thời điểm nó depart
            "enterTime",  # thời điểm nó chạm biên (vào TAZ mới)
            "fromTaz",    # TAZ gốc
            "toTaz",      # TAZ đích (theo attr trong <vehicle>)
            "fromRegion", # = fromTaz (vì region = taz)
            "toRegion",   # region đích = toTaz
            "borderEdge", # edge mà nó chạm vào biên
            "vType"       # loại phương tiện
        ])

    # Các dict lưu trạng thái
    veh_origin_taz: dict[str, str] = {}   # vehID -> fromTaz
    veh_depart_time: dict[str, float] = {}# vehID -> departTime
    veh_logged: dict[str, bool] = {}      # vehID -> đã log crossing?

    # SUMO command
    sumo_cmd = [
        checkBinary("sumo"),
        "-n", NET_XML,
        # "--additional-files", VTYPES_XML,    # Nếu cần dùng thêm vtypes
        "-r", ROUTES_INTER_ALL,
        "--no-step-log", "true",
        "--step-length", "1.0",
        "--start", "true"
    ]
    print("[RUN ]", " ".join(sumo_cmd))
    traci.start(sumo_cmd)

    sim_time = 0
    while sim_time <= SIM_END:
        traci.simulationStep()

        # 3.4.1: Lấy danh sách xe vừa depart, lưu depart_time và fromTaz
        for vehID in traci.simulation.getDepartedIDList():
            from_taz = traci.vehicle.getParameter(vehID, "fromTaz")
            to_taz   = traci.vehicle.getParameter(vehID, "toTaz")
            if from_taz is None or to_taz is None:
                continue
            veh_origin_taz[vehID] = from_taz
            veh_depart_time[vehID] = sim_time  # lưu thời điểm depart = sim_time
            veh_logged[vehID] = False

        # 3.4.2: Duyệt toàn bộ xe đang chạy
        for vehID in traci.vehicle.getIDList():
            # Nếu đã log crossing, bỏ qua
            if veh_logged.get(vehID, False):
                continue

            current_edge = traci.vehicle.getRoadID(vehID)
            if current_edge not in edge_to_taz:
                # Chưa chạm biên
                continue

            entered_taz = edge_to_taz[current_edge]
            origin_taz  = veh_origin_taz.get(vehID)
            # Nếu indeed crossing: entered_taz ≠ origin_taz
            if origin_taz is not None and entered_taz != origin_taz:
                enter_time = sim_time
                vtype      = traci.vehicle.getTypeID(vehID)
                depart_time= veh_depart_time.get(vehID, 0.0)
                to_taz     = traci.vehicle.getParameter(vehID, "toTaz") or ""

                with open(CROSSING_LOG_CSV, "a", newline="") as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([
                        vehID,
                        f"{depart_time:.2f}",
                        f"{enter_time:.2f}",
                        origin_taz,
                        to_taz,
                        origin_taz,    # fromRegion = origin_taz
                        entered_taz,   # toRegion = entered_taz
                        current_edge,
                        vtype
                    ])
                veh_logged[vehID] = True

        sim_time += 1

    traci.close()
    print("[DONE] Inter-region kết thúc. Log đã ghi vào:", CROSSING_LOG_CSV)


if __name__ == "__main__":
    run_inter_region()


[INFO] load_edge_to_taz: Đã đọc 81242 biên từ TAZ_XML
[RUN ] /home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/site-packages/sumo/bin/sumo -n /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml -r /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/new-strategy/routes_inter_all.xml --no-step-log true --step-length 1.0 --start true
 Retrying in 1 seconds
